# 스파크 스트리밍 실습 6교시 : 스트림 조인

> 

## 학습 목표
* 
  - 


## 목차
* [1. 스트리밍 조인](#1.-스트리밍-조인)
  - [1.1 스트림과 정적 테이블 조인](#1.1.-스트림과-정적-테이블-조인)
  - [1.2 스트림과 스트림 테이블 조인](#1.2.-스트림과-스트림-테이블-조인)
  - [1.3 워터마킹을 이용한 아우터 조인](#1.3-워터마킹을-이용한-아우터-조인)


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

# 현재 기동된 스파크 애플리케이션의 포트를 확인하기 위해 스파크 정보를 출력합니다
spark

21/08/17 14:24:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/17 14:24:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/08/17 14:24:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/08/17 14:24:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# 스트림 테이블을 주기적으로 조회하는 함수 (name: 이름, sql: Spark SQL, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStream(name, sql, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)              # 출력 Cell 을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Query: '+sql)
        display(spark.sql(sql))              # Spark SQL 을 수행합니다
        sleep(sleep_secs)                    # sleep_secs 초 만큼 대기합니다
        i += 1

# 스트림 쿼리의 상태를 주기적으로 조회하는 함수 (name: 이름, query: Streaming Query, iterations: 반복횟수, sleep_secs: 인터벌)
def displayStatus(name, query, iterations, sleep_secs):
    from time import sleep
    i = 1
    for x in range(iterations):
        clear_output(wait=True)      # Output Cell 의 내용을 지웁니다
        display('[' + name + '] Iteration: '+str(i)+', Status: '+query.status['message'])
        display(query.lastProgress)  # 마지막 수행된 쿼리의 상태를 출력합니다
        sleep(sleep_secs)            # 지정된 시간(초)을 대기합니다
        i += 1

<br>

## 1. 스트리밍 조인

> 스트리밍 데이터 집합간의 조인에 대해 학습합니다. 어떤 조인(이너, 아우터)이 지원되며, 스테이트풀 조인을 위해 저장되는 '상태'를 워터마크를 이용하여 제한하는 방법에 대해 배웁니다.

### 1.1 스트림과 `정적 테이블 조인`

> 데이터 로그에 모든 정보를 다 담을 수 없기 때문에, 로그의 타입과 그 정보를 가진 맵핑 테이블을 통해 집계 정보를 통해 집계 정보를 보여줄 수 있습니다. 광고 클릭 로그(stream)와 광고 당 발생하는 노출정보(static)를 이용하여 실시간 수익을 계산할 수 있습니다. 즉, 자주 변경되지 않으며 사전에 계획된 광고 노출 데이터(static)와의 조인이 필요한 경우에 사용하면 상대적으로 성능과 스트리밍의 효과를 누릴 수 있습니다

```python
impressionStatic = spark.read.parquet("/static/impression/dt=20210707")
clickStream = spark.readStream.format("kafka").load()
# 광고 구분자를 조인 키로 조인을 수행합니다 - stream inner-join static
matched = clickStream.join(impressionStatic, "adId")
# 혹은 stream left-outer-join 혹은 right-outer-join stream 만 지원합니다
matchedOuter = clickStream.join(impressionStatic, "adId", "leftOuter")
```

#### 1.1.1 스트림 조인에서 유의해야 할 사항
* 스트림-정적 조인의 경우 스테이트리스 연산이므로, '워터마킹'을 필요로 하지 않습니다
  - 워터마크는 스트림 처리의 중간 결과 상태 데이터를 말합니다
* 정적 데이터의 경우 반복적으로 읽게 되므로 캐싱하여 성능을 높일 수 있습니다
* 정적 데이터가 변경되는 경우 소스의 종류에 따라 반영되지 않을 수 있습니다
  - 파일 소스에 데이터가 추가(append)되는 경우 스트리밍 쿼리가 재시작 되기 전까지는 반영되기 어렵습니다


In [12]:
queryName = "staticJoin"
userPath = f"{work_dir}/data/user-static"
userStatic = spark.read.json(userPath)
userStatic.printSchema()

logsPath = f"{work_dir}/data/tumbling-stream"
# logsStream = spark.read.json(logsPath)
# logsStream.printSchema()
logsSchema = (
    StructType()
    .add(StructField("emp_id", LongType()))
    .add(StructField("emp_name", StringType()))
    .add(StructField("time", StringType()))
    .add(StructField("timestamp", LongType()))
)
logsStream = spark.readStream.format("json").schema(logsSchema).load(logsPath)

staticSelector = logsStream.join(userStatic, logsStream.emp_id == userStatic.user_id, "leftOuter")
staticWriter = staticSelector.writeStream.format("console").outputMode("append")

checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
staticTrigger = staticWriter.trigger(processingTime="5 second").option("checkpointLocation", checkpointLocation)

staticQuery = staticTrigger.start()
staticQuery.awaitTermination(10)
staticQuery.stop()

root
 |-- gender: string (nullable = true)
 |-- reg_dt: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_name: string (nullable = true)

-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------------+----------------------+----------+------+----------------------+-------+----------+
|emp_id|        emp_name|                  time| timestamp|gender|                reg_dt|user_id| user_name|
+------+----------------+----------------------+----------+------+----------------------+-------+----------+
|     1|        엘지전자|2016. 3. 20. 오후 1...|1458444054|  남성|2014. 1. 10. 오후 1...|      1|    김엘지|
|     1|        엘지전자|2016. 3. 20. 오후 1...|1458444055|  남성|2014. 1. 10. 오후 1...|      1|    김엘지|
|     1|        엘지전자|2016. 3. 20. 오후 1...|1458444056|  남성|2014. 1. 10. 오후 1...|      1|    김엘지|
|     1|        엘지전자|2016. 3. 20. 오후 3...|1458454057|  남성|2014. 1. 10. 오후 1...|      1|    김엘지|
|     2|        엘지화학|2016. 3. 20


<br>

### 1.2 스트림과 `스트림 테이블 조인`

> 스트림과 스트림 데이터집합의 조인의 경우 양쪽 스트림 모두 지연될 수 있기 때문에 완전하지 않은 데이터 소스에 대한 조인이 일어날 수 있으므로 주의해서 다루어야 합니다. 이번에는 click 뿐만 아니라 impression 까지도 스트림 데이터 소스로 조인하는 예제를 학습합니다

![figure.8-11](images/figure.8-11.png)

<br>

#### 1.2.1 스트림 간의 조인의 특징

* 엔진은 소스가 스트림 - 스트림 임을 인지하고, impression-and-click 정보의 상태를 버퍼링합니다
* 버퍼링 과정에서 서로 매칭되는 데이터를 통해 조인 연산이 수행됩니다. 코드와 시각화한 화면은 아래와 같습니다.

```python
impressions = spark.readStream.format("kafka").option("...").load()
clicks = spark.readStream.format("kafka").option("...").load()
matched = clicks.join(impressions, "adId")
```


In [28]:
queryName = "streamStreamJoin"
impressionSchema = StructType().add(StructField("adId", LongType())).add(StructField("impressionTime", TimestampType())).add(StructField("time", StringType()))
clickSchema = StructType().add(StructField("adId", LongType())).add(StructField("clickTime", TimestampType())).add(StructField("time", StringType()))

impressions = spark.readStream.format("json").schema(impressionSchema).load(f"{work_dir}/data/impressions")
imp = impressions.selectExpr("adId AS impressionAdid", "impressionTime").withWatermark("impressionTime", "2 hours") # impressionsWithWatermark

clicks = spark.readStream.format("json").schema(clickSchema).load(f"{work_dir}/data/clicks")
clk = clicks.selectExpr("adId AS clickAdid", "clickTime").withWatermark("clickTime", "3 hours") # clicksWithWatermark

joinCondition = "impressionAdid = clickAdid AND clickTime BETWEEN impressionTime AND impressionTime + interval 1 hour"
streamStreamSelector = impressionsWithWatermark.join(clicksWithWatermark, joinCondition)

# streamStreamSelector = clicks.join(impressions, clicks.adId == impressions.adId, "inner")
streamStreamWriter = streamStreamSelector.writeStream.format("memory").queryName(queryName).outputMode("append")
checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
streamStreamJoinQuery = streamStreamWriter.option("checkpointLocation", checkpointLocation).start()
streamStreamJoinQuery.awaitTermination(10)
streamStreamJoinQuery.stop()

AnalysisException: USING column `impressionAdid = clickAdid AND clickTime BETWEEN impressionTime AND impressionTime + interval 1 hour` cannot be resolved on the left side of the join. The left-side columns: [impressionAdid, impressionTime]


![figure.8-12](images/figure.8-12.png)

> 그림에서의 파란색 점이 impression 과 click 의 '이벤트 타임'을 나타내고 있으며, '마이크로 배치'를 넘나드는 매칭되는 데이터 집합을 통해 조인연산이 일어나게 됩니다.

<br>

#### 1.2.2 스트리밍 간의 조인의 가정

* 이러한 '이벤트 타임' 기준 시간으로 조인이 이루어지기 때문에 두 이벤트는 같은 시간 "same wall clock time" 으로 저장되어야 문제가 없습니다
* 위의 이벤트 처리에서는 대기 시간을 지정하지 않았기 때문에 조인되지 않은 데이터의 '스트리밍 상태'는 계속 대기하게 됩니다


<br>

#### 1.2.3 스트리밍 간의 조인 처리시에 고려할 사항

* 광고노출 이후 클릭 사이에 소요되는 최대 시간은 얼마인가?
  - 최대 0초(바로 클릭)에서 최대 한 시간까지 대기했다가 클릭이 발생할 수 있다고 가정해 보았습니다
* 광고의 노출 및 클릭 정보가 네트워크 장애 등의 이슈로 얼마나 지연되어 수신될 수 있는가?
  - 노출 및 클릭이 각 각 2시간, 3시간까지 지연 될 수 있다고 가정해 보았습니다

> 위에서 얘기한 '이벤트 타임' 제약이나 지연 등의 조건은 워터마크와 시간 범위 조건을 이용하여 데이터프레임 연산에 적용할 수 있습니다. 

* 1. 두 가지 '데이터 소스'는 얼마나 지연될 수 있는지 '워터마크'를 통해 정의합니다
* 2. 두 가지 '데이터 소스'간의 '이벤트 타임'에 대한 제약을 정의합니다
  - 오래된 입력 로우가 언제 필요 없어지는 지에 대한 시점을 엔진이 인지하게 되는 제약 조건을 말합니다

* 예제를 통해서 정의해 보겠습니다
  - Time range join conditions : 노출 이후에 클릭이 최대 1시간 정도까지만 발생할 수 있다고 가정했으므로
    * "leftTime BETWEEN rightTime AND rightTime + INTERVAL 1 HOUR"
    * 클릭 시간이 노출 시간 + 1시간 까지의 시간에 대해서 조인될 수 있습니다
  - Join on event-time windows : 
    * leftTimeWindow = rightTimeWindow
    * 클릭의 윈도우 시간은 3시간, 노출은 2시간으로 두었으므로 그 시간 범위에 대해 조인될 수 있습니다



![figure.8-13](images/figure.8-13.png)

<br>

#### 1.2.4 그림을 통해 2가지 스트림에 대한 가정을 분석합니다

* 1. 노출 데이터를 기준 : 노출은 2시간 지연이고, 클릭은 3시간 지연이므로, 노출 이후 1시간 클릭 가정을 고려하면 최대 4시간 노출 버퍼가 유지되어야 합니다
* 2. 클릭 데이터를 기준 : 유사하게, 3시간 짜리 클릭 윈도우는 2시간의 버퍼만 유지되더라도 매칭이 가능합니다

```python
impressionsWithWatermark = (
    impressions
    .selectExpr("adId AS impressionAdid", "impressionTime")
    .withWatermark("impressionTime", "2 hours")
)
clickWithWatermark = (
    clicks
    .selectExpr("adId AS clickAdid", "clickTime")
    .withwatermark("clickTime", "3 hours")
)
joined = impressionsWithWatermark.join(
    clickWithWatermark,
    exp("""
        clickAdId = impressionAdid AND
        clickTime BETWEEN impressionTime AND impressionTime + interval 1 hour
        """)
)
```

<br>

#### 1.2.5 Inner Join 연산에 대한 기억해 두어야하는 키포인트

* Inner Join 의 경우, 워터마킹과, 이벤트 타임 제약 두 가지 모두 선택 사항이므로, 'Unbounded State' 에 대한 위험성을 내포하고 있습니다. 즉, 두 가지 옵션을 지정해 주었을 때에만 더 이상 조인되지 않을 수 있는 '상태'가 정리 대상으로 관리된다는 의미입니다
* 워터마킹 집계 연산에서 다루었던 것과 동일하게, 워터마킹에 의해 지정된 시간 범위내에 지연된 로그가 누락되지 않음을 보장하지만, 지연 로그가 처리되지 않는 것은 보장하지는 않습니다.



<br>


### 1.3 *워터마킹*을 이용한 `아우터 조인`

> 위의 Inner Join 예제의 경우 노출은 되더라도 클릭되지 않은 경우는 전혀 리포팅 되지 않게 됩니다. 그래서 노출은 되지만 클릭되지 않은 경우를 고려하기 위해서 Outer Join 을 사용하게 됩니다

```python
(
    impressionsWithWatermark.join(
        clickWithWatermark,
        expr("""
        clickAdId = impressionAdId AND
        clickTime BETWEEN impressionTime AND impressionTime + interval 1 hour
        """),
        "leftOuter"
    )
)
```

#### 1.3.1 Outer Join 의 특징
* 예상대로 매 '마이크로 배치'의 노출에 대한 값에 대해 클릭이 없더라도 출력됩니다
  - Inner Join 의 경우, 워터마킹에 대한 정보가 없고, 제약 시간 조건만 있어도 클릭로그를 처리하지 않는 시점을 인터벌 시간이 지난 클릭로그 시간만으로 인지할 수 있으나,
  - Outer Join 의 경우, '노출 로그'에 매 '마이크로 배치' 시간 마다 NULL 로 채워진 클릭 로그에 대한 결과를 출력해야만 합니다 (지연일 수도, 클릭을 안 한 것일 수도 있으므로), 만약 명시되지 않는 경우 해당 NULL 값을 채워줄 수 없기 때문에 해당 타임 테이블의 출력 시점을 정할 수 없기 때문에 반드시 '워터마킹'과 '인터벌' 정보에 대한 제약을 반드시 명시되어야만 합니다
